In [0]:
# Databricks Widgets for Dynamic Control
dbutils.widgets.text("target_company", "Aura Align", "Select Company")
dbutils.widgets.text("campaign_limit", "200", "Max Campaigns")

# Read Widget Values
target_company = dbutils.widgets.get("target_company")
campaign_limit = int(dbutils.widgets.get("campaign_limit"))

print(f"Target Company: {target_company}")
print(f"Campaign Limit: {campaign_limit}")


In [0]:
from pyspark.sql.functions import col

df = spark.table("adwiseai.ads_dataset_cleaned")

# Filter by company and limit campaign count
df_company = df.filter(col("Company") == target_company).limit(campaign_limit)

display(df_company)


In [0]:
from pyspark.sql import functions as F

channel_summary = df_company.groupBy("Channel_Lower").agg(
    F.count("*").alias("campaign_count"),
    F.avg("ROI").alias("avg_ROI"),
    F.avg("CTR").alias("avg_CTR"),
    F.avg("CPC").alias("avg_CPC"),
    F.avg("Conversion_Rate").alias("avg_Conversion_Rate"),
    F.avg("Engagement_Score").alias("avg_Engagement_Score")
).orderBy("avg_ROI", ascending=False)

display(channel_summary)

# Convert to JSON-style dict
channel_data = channel_summary.toPandas().to_dict(orient="records")


In [0]:
import json

prompt_path = "/Workspace/Users/njdejong99@gmail.com/adwiseai/prompts/prompt1.json"

with open(prompt_path, "r") as f:
    prompt_templates = json.load(f)

template = prompt_templates["channel_comparison_with_strategy"]

prompt = template.format(
    company=target_company,
    channel_data=json.dumps(channel_data, indent=2)
)


In [0]:
from openai import AzureOpenAI

# Azure OpenAI configuration
endpoint = "https://forfunz.openai.azure.com/"
subscription_key = dbutils.secrets.get(scope="LLMops vault", key="OpenAI-AzureKey1")  # Databricks secret

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

# Send a chat completion request to get insights
response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are an expert digital advertising analyst."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model="gpt35-insights"  # Use the deployed model
)

# Extract and display the insights from the response
channel_insight = response.choices[0].message.content
print(channel_insight)


In [0]:
from datetime import datetime

# Prepare data for logging
insight_df = spark.createDataFrame(
    [(target_company, datetime.now(), prompt, channel_insight)],
    ["Company", "Timestamp", "Prompt", "Insight"]
)

# Save the insight to the Delta table
insight_df.write.mode("append").format("delta").saveAsTable("adwiseai.insight_by_channel")
